In [1]:
from holdup.parser.replayable_hand import ReplayableHand, Streets

In [2]:
split_hands = """STATE:0:r241f:8h4c|6c5h:-100|100:act1_2pn_2016|hugh_2pn_2016
        STATE:1:r223f:6h9c|Jc8h:-100|100:hugh_2pn_2016|act1_2pn_2016
        STATE:2:r241r675c/cr1629c/cc/r2606c:QcAc|TsAs/JsTh7d/Qh/Td:-2606|2606:act1_2pn_2016|hugh_2pn_2016
        STATE:3:cc/r241c/r581f:2dQh|6d3s/Jc4h2c/Tc:241|-241:hugh_2pn_2016|act1_2pn_2016
        STATE:4:r241r809c/cr1953c/cr4714f:8cTc|9sAd/3cAc3s/Js:-1953|1953:act1_2pn_2016|hugh_2pn_2016
        STATE:5:r273f:9d4c|JdTd:-100|100:hugh_2pn_2016|act1_2pn_2016
        STATE:6:r241r675c/cr1629c/cc/cc:9c9s|7s7d/AcAhKh/5c/Qc:1629|-1629:act1_2pn_2016|hugh_2pn_2016
        STATE:7:r273f:4d3d|KdQh:-100|100:hugh_2pn_2016|act1_2pn_2016
        STATE:8:r241f:7c5s|6h6s:-100|100:act1_2pn_2016|hugh_2pn_2016""".split("\n")

hand0 = ReplayableHand(split_hands[0]).matrix_next_action_by_street()
hands = [ReplayableHand(h).matrix_next_action_by_street() for h in split_hands]

In [7]:
print(f"Each hand is now a dictionary, keyed by street with a list of actions: {type(hands[0][int(Streets.Preflop)])}")
print(f"Each list item now contains the matrix and a tuple that indicates whether" + \
       f"the next action crosses a round boundary and what the action is: {hands[0][int(Streets.Preflop)][0]}")

Each hand is now a dictionary, keyed by street with a list of actions: <class 'list'>
Each list item now contains the matrix and a tuple that indicates whetherthe next action crosses a round boundary and what the action is: (array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0.],

In [27]:
import random
import os
from typing import Tuple, List
import functools

import numpy as np

"""
- Last possible action to final action
tuple(pf, f, t, r)
preflop: List[Matrix, Integer: Supervised Target]
flop: List[Matrix, Integer: Supervised Target]
turn: List[Matrix, Integer: Supervised Target]
river: List[Matrix, Integer: Supervised Target]

- Every possible action to subsequent action; not across round boundaries
preflop: List[Matrix, Integer: Supervised Target]
flop: List[Matrix, Integer: Supervised Target]
turn: List[Matrix, Integer: Supervised Target]
river: List[Matrix, Integer: Supervised Target]
"""
def reducer(reduce_type: str):
    def doit(agg: Tuple[
        List[Tuple[np.ndarray, Tuple[bool, int]]],
        List[Tuple[np.ndarray, Tuple[bool, int]]],
        List[Tuple[np.ndarray, Tuple[bool, int]]],
        List[Tuple[np.ndarray, Tuple[bool, int]]]
        ], hands_dict):

        
        if reduce_type == "last_possible":
            # Get the 2nd to last action
            appender = lambda street, actions: street + [actions[-2]] if len(actions) > 1 else street
        else:
            # Get all actions up to the last action
            appender = lambda street, actions: street + actions[0:-1] if len(actions) > 1 else street

        return [
            appender(actions_for_training, hands_dict[street_name])
            for street_name, actions_for_training in zip(
                (Streets.Preflop, Streets.Flop, Streets.Turn, Streets.River),
                agg
            )
        ]
    
    return doit


def get_test_file_lines():
    # To verify for yourself, copy some hands to a file in some directory and set it here
    # Then use the csv file enumeration
    data_dir = r"/home/david/development/omscs/dl/CS7643_final_project/src/holdup/data/test"

    # Iterate over log (NOT CSV!!) files; log files are RAW
    csv_files = [data_dir + "/" +f for f in os.listdir(data_dir) if f.endswith('.log')]
    random.shuffle(csv_files)
    return [
        [open(csv_file, 'r').readlines()]
        for csv_file in csv_files
    ]


def get_provided_examples():
    return [
        """STATE:0:cc/cr200f:3c8s|5c7d/ThKhTc:-100|100:Rembrant|SartreNL
        STATE:1:cr300f:As9s|8h6c:100|-100:SartreNL|Rembrant
        STATE:2:cc/cr200f:Kc7d|2d8d/TsAc6c:-100|100:Rembrant|SartreNL
        STATE:3:r400f:2c5h|AhTd:-100|100:SartreNL|Rembrant
        STATE:4:r300f:8s4c|5dAd:-100|100:Rembrant|SartreNL""".split("\n")
    ]

In [54]:
def demonstrate_dataset_creation(last_possible: bool) -> None:
    per_street_actions = []

    # Valid values: "last_possible", if not this, assuming you want all actions up to last per street
    reduce_type = "last_possible" if last_possible else "lolololl"

    # You can switch to get_test_file_lines() if you want to reading a directory locally
    for hand_group in get_provided_examples():
        data = [ReplayableHand(h).matrix_next_action_by_street() for h in hand_group]
        per_street_actions.append(functools.reduce(reducer(reduce_type), data, ([], [], [], [])))

    # First index = file ("hand group") index, second = street, third = Actions (ie: List[Tuple[matrix, Tuple[bool, int]]])
    print("Some return type info")
    print(len(per_street_actions))
    print(len(per_street_actions[0]))
    print(len(per_street_actions[0][0]))

    print("Next actions (either from each penultimate preflop state, or from all non-final preflop states)")
    preflop = [
        # This is the data structure we agreeged upon.
        # Matrix, Next Action
        (action[0], action[1][1]) 
        for file in range(len(per_street_actions))
        for action in per_street_actions[file][0] # Street set here
    ]

    # Verify next actions.
    # For the example, these are all correct. You should verify to ensure you understand why/how.
    for i in range(len(preflop)):
        print(preflop[i][1])

    print("Next actions (either from each penultimate flop state, or from all non-final flop states)")
    flop = [
        (action[0], action[1][1]) 
        for file in range(len(per_street_actions))
        for action in per_street_actions[file][1]
    ]

    # For the example, only two hands went to the flop
    # STATE:0:cc/cr200f:3c8s|5c7d/ThKhTc:-100|100:Rembrant|SartreNL
    # STATE:2:cc/cr200f:Kc7d|2d8d/TsAc6c:-100|100:Rembrant|SartreNL
    for i in range(len(flop)):
        print(flop[i][1])

    print("This conclues all hands from provided examples. 3 indicates the next action is fold")

In [55]:
last_possible = True
demonstrate_dataset_creation(last_possible)

Some return type info
1
4
5
Next actions (either from each penultimate preflop state, or from all non-final preflop states)
0
3
0
3
3
Next actions (either from each penultimate flop state, or from all non-final flop states)
3
3
This conclues all hands from provided examples. 3 indicates the next action is fold


In [56]:
last_possible = False
demonstrate_dataset_creation(last_possible)

Some return type info
1
4
6
Next actions (either from each penultimate preflop state, or from all non-final preflop states)
0
2
3
0
3
3
Next actions (either from each penultimate flop state, or from all non-final flop states)
2
3
2
3
This conclues all hands from provided examples. 3 indicates the next action is fold


In [ ]:
last_possible = True
demonstrate_dataset_creation(last_possible)

Some return type info
1
4
5
Next action following each penultimate preflop state
0
3
0
3
3
Next action following each penultimate flop state
3
3
This conclues all hands from provided examples. 3 indicates the next action is fold


0
3
0
3
3


3
3


# Legacy below

Changes have been made and the below code wasn't updated. If it's important you someone, it shouldn't be hard to update it.

In [3]:
print(f"This hand goes raise fold. The next action is fold: {hands[0][int(Streets.Preflop)][0][1]}")
print(f"and then there are no more actions: {hands[0][int(Streets.Preflop)][1][1]}")
print(f"The same for the next hand: {','.join([str(hands[1][int(Streets.Preflop)][n][1]) for n in range(2)])}")
print("*" * 10 + f"\n{split_hands[2]}")
print(f"Next hand goes raise, raise, call: {','.join([str(hands[2][int(Streets.Preflop)][n][1]) for n in range(3)])}")
print(f"check, raise, call: {','.join([str(hands[2][int(Streets.Flop)][n][1]) for n in range(3)])}")
print(f"check, check: {','.join([str(hands[2][int(Streets.Turn)][n][1]) for n in range(2)])}")
print(f"raise, call: {','.join([str(hands[2][int(Streets.River)][n][1]) for n in range(2)])}")

This hand goes raise fold. The next action is fold: (False, 3)
and then there are no more actions: (True, -1)
The same for the next hand: (False, 3),(True, -1)
**********
        STATE:2:r241r675c/cr1629c/cc/r2606c:QcAc|TsAs/JsTh7d/Qh/Td:-2606|2606:act1_2pn_2016|hugh_2pn_2016
Next hand goes raise, raise, call: (False, 2),(False, 1),(True, 0)
check, raise, call: (False, 2),(False, 1),(True, 0)
check, check: (False, 0),(True, 2)
raise, call: (False, 1),(True, -1)


In [4]:
print("You can play with changing the street and action indices to see how the output matrix changes")
hands[2][int(Streets.Flop)][2][0][0:8,13:17]

You can play with changing the street and action indices to see how the output matrix changes


array([[0., 1., 1., 0.],
       [1., 1., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [13]:
# After the first action, only actions recorded are for player A, who bet
print(hand0[0][0:4,13:17])
hand0[0][4:8,13:17]

[[0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


array([[0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [14]:
# After the second, we see player B's action recorded
print(hand0[1][0:4,13:17])
hand0[1][4:8,13:17]

[[0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


array([[0., 0., 0., 1.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.],
       [0., 0., 0., 0.]])

In [3]:
# This one went to the river, so we can see different streets and track hand matrices
hand2 = ReplayableHand(hands[2]).generate_matrices()

In [6]:
# Player A has AcQc and community cards are JsTh7d/Qh/Td
# Show the evolution of player A's hand matrix across each round
# This hand had 3 actions preflop, 3 actions flop, two actions turn, two actions river
print(hand2[0][0:4,0:13])
print(hand2[3][0:4,0:13])
print(hand2[6][0:4,0:13])
print(hand2[8][0:4,0:13])

[[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
[[1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]


In [4]:
# Show the evolution of action matrices across rounds. You'll need to open in a text editor if running in VS code
for i in range(len(hand2)):
    print("*" * 20)
    print(hand2[i][0:8,13:17])

********************
[[0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
********************
[[0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
********************
[[0. 1. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
********************
[[0. 1. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
********************
[[0. 1. 1. 0.]
 [0. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
********************
[[0. 1. 1. 0.]
 [1. 1. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
********************
[[0. 1. 1. 0.]
 [1. 1. 0. 0.]
 [1. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]
******